In [2]:
!pip -q install torchsde==0.2.6 einops tqdm pyyaml gdown
!pip -q install git+https://github.com/fra31/auto-attack

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [3]:
!git clone https://github.com/NVlabs/DiffPure tmp_diffpure


Cloning into 'tmp_diffpure'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 161 (delta 60), reused 150 (delta 49), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 659.99 KiB | 14.35 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [4]:
%%bash
mv tmp_diffpure/* .
mv tmp_diffpure/.[!.]* . 2>/dev/null || true
rmdir tmp_diffpure

In [5]:
import torch, sys, platform
print("python:", sys.version)
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))


python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
torch: 2.9.0+cu126
cuda available: True
gpu: Tesla T4


In [6]:
!pip -q install gdown
!mkdir -p pretrained/score_sde

# remove the incorrect tiny HTML file
!rm -f pretrained/score_sde/checkpoint_8.pth

# download by file id
!gdown --id 16_-Ahc6ImZV5ClUc0vM5Iivf8OJ1VSif -O pretrained/score_sde/checkpoint_8.pth


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=16_-Ahc6ImZV5ClUc0vM5Iivf8OJ1VSif
From (redirected): https://drive.google.com/uc?id=16_-Ahc6ImZV5ClUc0vM5Iivf8OJ1VSif&confirm=t&uuid=c8fa1af2-4b07-42f7-834b-df57297ad6a7
To: /content/pretrained/score_sde/checkpoint_8.pth
100% 1.71G/1.71G [00:25<00:00, 68.0MB/s]


In [11]:
from utils import get_image_classifier
clf = get_image_classifier("cifar10-wideresnet-28-10")
print(clf.__class__)
print(clf)

using cifar10 wideresnet-28-10...


Downloading...
From (original): https://drive.google.com/uc?id=1t98aEuzeTL8P7Kpd5DIrCoCL21BNZUhC
From (redirected): https://drive.google.com/uc?id=1t98aEuzeTL8P7Kpd5DIrCoCL21BNZUhC&confirm=t&uuid=4d8b76d5-b983-46db-b77a-1a168f717854
To: /content/models/cifar10/Linf/Standard.pt
100%|██████████| 292M/292M [00:04<00:00, 71.8MB/s]


<class 'robustbench.model_zoo.architectures.wide_resnet.WideResNet'>
WideResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (block1): NetworkBlock(
    (layer): Sequential(
      (0): BasicBlock(
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(16, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(160, 160, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (convShortcut): Conv2d(16, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BasicBlock(
        (bn1): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(160, 160, kernel_size=(3, 3

In [10]:
pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 14.7 MB/s eta 0:00:00


In [ ]:
!CUDA_VISIBLE_DEVICES=0 python eval_sde_adv.py \
  --exp ./exp_results \
  --config cifar10.yml \
  -i cifar10-diffpure-Linf-standard \
  --t 25 \
  --adv_eps 0.031373 \
  --adv_batch_size 64 \
  --num_sub 64 \
  --domain cifar10 \
  --classifier_name cifar10-wideresnet-28-10 \
  --seed 0 \
  --data_seed 0 \
  --diffusion_type sde \
  --score_type score_sde \
  --attack_version standard


INFO - eval_sde_adv.py - 2025-12-14 18:28:42,447 - Using device: cuda
ngpus: 1, adv_batch_size: 64
starting the model and loader...
using cifar10 wideresnet-28-10...
diffusion_type: sde
model_config: Namespace(data=Namespace(dataset='CIFAR10', category='cifar10', image_size=32, num_channels=3, random_flip=True, centered=True, uniform_dequantization=False), model=Namespace(sigma_min=0.01, sigma_max=50, num_scales=1000, beta_min=0.1, beta_max=20.0, dropout=0.1, name='ncsnpp', scale_by_sigma=False, ema_rate=0.9999, normalization='GroupNorm', nonlinearity='swish', nf=128, ch_mult=[1, 2, 2, 2], num_res_blocks=8, attn_resolutions=[16], resamp_with_conv=True, conditional=True, fir=False, fir_kernel=[1, 3, 3, 1], skip_rescale=True, resblock_type='biggan', progressive='none', progressive_input='none', progressive_combine='sum', attention_type='ddpm', init_scale=0.0, embedding_type='positional', fourier_scale=16, conv_size=3), training=Namespace(sde='vpsde', continuous=True, reduce_mean=True, n_